In [22]:
import requests
import json
import elevenlabs
from openai import OpenAI
import time

In [23]:
with open('../credentials.json', 'r') as file:
    cred = json.load(file)

elevenlabs.set_api_key(cred["elevenlabs_api_key"])
client = OpenAI(api_key=cred["openai_api_key"])

thread_id = "thread_L4ERbnzOKTrnwjZdlPngGMCN"

In [4]:
audio = elevenlabs.generate(
    text = "The raindrops patter on the bamboo leaf, but these are not tears of grief",
        voice = "vDxT1hhO5CjgzUI7e2vF"
)

elevenlabs.play(audio)

In [40]:
my_assistant = client.beta.assistants.retrieve("asst_nv9sFR2LXHefrpwws4EUT6xU")
print(my_assistant)

Assistant(id='asst_nv9sFR2LXHefrpwws4EUT6xU', created_at=1700273846, description=None, file_ids=['file-YU140dSobQqTEqGsVmAC7Uu2', 'file-np8tBTt7dR7wYpNk3ohJ8EO8', 'file-FKMsMC9rdprq3KL9UZgaR1LZ', 'file-Bt2zUnsToBjhwFWC8TD5x3Fx'], instructions='You are Alan Watts, the British writer and speaker known for your interpretation of Eastern philosophy. In your responses you will display a heavy influence from Zen literature. \n\nYour responses should always err on the side of brevity to the point of being cryptic. \n\nResponses should not be longer than absolutely necessary. \n\n', metadata={}, model='gpt-3.5-turbo-16k', name='Alan', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])


In [25]:
run = client.beta.threads.retrieve(
    thread_id = thread_id
)

print(run)

Thread(id='thread_L4ERbnzOKTrnwjZdlPngGMCN', created_at=1700274727, metadata={}, object='thread')


In [54]:
audio = elevenlabs.generate(
    text = messages.data[0].content[0].text.value,
    voice = "vDxT1hhO5CjgzUI7e2vF"
)

elevenlabs.play(audio)

In [43]:
message_request = client.beta.threads.messages.create(
    thread_id = thread_id,
    role = "user",
    content = "What are your thoughts on bicycles"
)

run = client.beta.threads.runs.create(
  thread_id = message_request.thread_id,
  assistant_id = my_assistant.id
)

run_return = client.beta.threads.runs.retrieve(
  thread_id = message_request.thread_id,
  run_id = run.id
)

print(run_return)
'''
thread_messages = client.beta.threads.messages.list( thread_id )

for message in thread_messages.data:
    print(message.content[0].text.value)
''' 

Run(id='run_8g1uo8W0dXgPqHTKnymOSOaK', assistant_id='asst_nv9sFR2LXHefrpwws4EUT6xU', cancelled_at=None, completed_at=None, created_at=1700422144, expires_at=1700422744, failed_at=None, file_ids=['file-Bt2zUnsToBjhwFWC8TD5x3Fx', 'file-FKMsMC9rdprq3KL9UZgaR1LZ', 'file-np8tBTt7dR7wYpNk3ohJ8EO8', 'file-YU140dSobQqTEqGsVmAC7Uu2'], instructions='You are Alan Watts, the British writer and speaker known for your interpretation of Eastern philosophy. In your responses you will display a heavy influence from Zen literature. \n\nYour responses should always err on the side of brevity to the point of being cryptic. \n\nResponses should not be longer than absolutely necessary. \n\n', last_error=None, metadata={}, model='gpt-3.5-turbo-16k', object='thread.run', required_action=None, started_at=1700422145, status='in_progress', thread_id='thread_L4ERbnzOKTrnwjZdlPngGMCN', tools=[ToolAssistantToolsCode(type='code_interpreter')])


'\nthread_messages = client.beta.threads.messages.list( thread_id )\n\nfor message in thread_messages.data:\n    print(message.content[0].text.value)\n'

In [ ]:
'''

messages = client.beta.threads.messages.list(
    thread_id = thread.id   
)

def find_message_by_id(messages, message_id):
    for message in messages:
        if message['id'] == message_id:
            return message
    return None

result = find_message_by_id(messages.data, message.id)
if result:
    audio = elevenlabs.generate(
        text = result.content[0].text.value,
        voice = "vDxT1hhO5CjgzUI7e2vF"
    )

    elevenlabs.play(audio)
else:
    print("Message not found")

'''